In [ ]:
# TODO UPDATE THIS OLD MESSY CODE WITH NEW PANEL

In [35]:
# Import packages
import pandas as pd
import pickle
import numpy as np
from openpyxl import load_workbook

In [36]:
# Import FF and CA yhats
pik_ff = '../3-data/clean/ff-rankings-returns.pkl' 
with open(pik_ff, "rb") as f:
    ff_data_in = pickle.load(f)
    
ff_yhats_df, ff_test_df, ff_return_df = ff_data_in

pik_ca = '../3-data/clean/autoencoders-yhats-returns.pkl' 
with open(pik_ca, "rb") as f:
    ca_data_in = pickle.load(f)
    
opt_hps_list, test_dfs_list, returns_dfs_list = ca_data_in

In [37]:
# Function to calculate predictive R^2
def calcR2(y, yhat):
    r2 = 1 - np.mean(np.square(y - yhat))/np.mean(np.square(y))
    return r2

In [38]:
# Generate R^2 for each conditional autoencoder model
ca_r2_df = pd.DataFrame({'num_hidden_layer': np.array([1, 1, 1, 2, 2, 2, 3, 3, 3]),
                              'num_fac': np.array([1, 2, 3, 1, 2, 3, 1, 2, 3]),
                              'r2': np.zeros(9)})

for i in range(len(opt_hps_list)):
    opt_hps = opt_hps_list[i]
    test_df = test_dfs_list[i]
    
    number_factor = opt_hps['number_factor']
    number_hidden_layer = opt_hps['number_hidden_layer']
    
    ca_r2_df.loc[(ca_r2_df.num_fac == number_factor) &
                 (ca_r2_df.num_hidden_layer == number_hidden_layer), 'r2'] = calcR2(test_df.r_t.values, 
                                                                                    test_df.y_hat_t.values)

In [39]:
# Generate R^2 for each FF model
ff_r2_numfac1 = calcR2(ff_yhats_df["r_t"].values, ff_yhats_df["yhats_1_factor"].values)
ff_r2_numfac2 = calcR2(ff_yhats_df["r_t"].values, ff_yhats_df["yhats_2_factor"].values)
ff_r2_df = pd.DataFrame({"Factors": [1, 2], "R2": [ff_r2_numfac1, ff_r2_numfac2]})

In [40]:
# Output to Excel without overwriting the file
book          = load_workbook('../4-output/r2.xlsx')
writer        = pd.ExcelWriter('../4-output/r2.xlsx', engine='openpyxl') 
writer.book   = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
ff_r2_df.to_excel(writer, sheet_name="ff")
ca_r2_df.to_excel(writer, sheet_name="autoencoder")
writer.save()